In [ ]:
import pandas as pd
pd.options.display.max_columns = None
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamgamelog

import numpy as np

In [ ]:

# Get all NBA teams
nba_teams = teams.get_teams()
nba_teams[0]

In [ ]:
# pd.DataFrame(nba_teams).to_csv("../data/team_info.csv", index=False)

In [ ]:
# Initialize a list to store all games dataframes
all_games = []

season_start = 2022
season_end = 2025
for season in range(season_start, season_end + 1):
    print(season)

    # Loop over all teams
    for team in nba_teams[:]:
        team_id = team["id"]
        # print(team["full_name"], )

        # Query the API
        response = teamgamelog.TeamGameLog(
            team_id=team_id,
            season=season,
            season_type_all_star='Regular Season'
        )

        games = response.get_data_frames()[0].sort_values(by="Game_ID")
        games['team'] = team['full_name']
        games['season_start_year'] = season
        # print(f"num games: {games.shape[0]}")
        
        # Append games dataframe to the list
        all_games.append(games)

    # Concatenate all games dataframes
    all_games_df = pd.concat(all_games)
    

print(all_games_df.shape)
all_games_df

In [ ]:
# opponents abbreviation
all_games_df['opponent_abbreviation'] = all_games_df['MATCHUP'].str[-3:]
all_games_df['team_abbreviation'] = all_games_df['MATCHUP'].str[:3]

# home or away
all_games_df['is_home_game'] = np.where(all_games_df['MATCHUP'].str.contains("@"), 0, 1)

# convert to date
all_games_df['GAME_DATE'] = pd.to_datetime(all_games_df['GAME_DATE'])

all_games_df.head()

In [ ]:
print(all_games_df.shape)
all_games_df

In [ ]:
# how many games per team per regular season
piv = all_games_df.pivot_table(
    index=["team", "season_start_year"],
    values=["Game_ID"],
    aggfunc='count'
)
print(piv.shape)
piv.head(30)

In [ ]:
# why does 2020 only have 72 games

In [ ]:
import time
# print(time.ctime())
# all_games_df.to_csv("../data/2013-2022-regular-season-matchups.csv", index=False)


In [ ]:
all_games_df.head(50)